# Sincronizador de base de datos
2025-10-25

In [7]:
import os
import requests

In [8]:
table = 'items'
url_remote = "https://www.multisits.com/"
local_url = "http://localhost/accn/"
url_generate_files = f'{url_remote}api/sync/generate_files/{table}'
path_download = f'D:\\xampp\\htdocs\\accn\\public\\content\\database\\{table}\\'

In [9]:
# Función para obtener la lista de archivos desde la URL dada
def get_files_list(url):
    files = []
    try:
        # Realiza la solicitud GET
        response = requests.get(url, timeout=15)

        # Verifica el código de estado
        response.raise_for_status()

        # Intenta parsear el JSON
        sync_response = response.json()
        files = sync_response['files']

        """ print("✅ Respuesta recibida correctamente:")
        print(sync_response) """

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Error HTTP: {http_err}")
        sync_response = None
    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Error de conexión o timeout: {req_err}")
        sync_response = None
    except ValueError:
        print("⚠️ La respuesta no es JSON válido.")
        print("Contenido recibido:")
        print(response.text)
        sync_response = None
    return files

In [13]:
# Generar los archivos y obtener la lista para descargar
files_list = get_files_list(url_generate_files)
print(url_generate_files)

# Recorrer files_list para descargar cada archivo
for file_name in files_list:
    url_download = f"{url_remote.rstrip('/')}/public/content/database/{table}/{file_name}"
    file_path = os.path.join(path_download, file_name)

    try:
        response = requests.get(url_download, timeout=15)
        response.raise_for_status()  # Verifica que la solicitud fue exitosa

        os.makedirs(path_download, exist_ok=True)

        with open(file_path, 'wb') as file:
            file.write(response.content)

        print(f"✅ Archivo descargado y guardado en: {file_path}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar el archivo: {e}")


https://www.multisits.com/api/sync/generate_files/items
✅ Archivo descargado y guardado en: D:\xampp\htdocs\accn\public\content\database\items\part_1.json
✅ Archivo descargado y guardado en: D:\xampp\htdocs\accn\public\content\database\items\part_2.json
✅ Archivo descargado y guardado en: D:\xampp\htdocs\accn\public\content\database\items\part_3.json
✅ Archivo descargado y guardado en: D:\xampp\htdocs\accn\public\content\database\items\part_4.json
✅ Archivo descargado y guardado en: D:\xampp\htdocs\accn\public\content\database\items\part_5.json


In [11]:
# Eliminar los archivos generados en el servidor remoto
url_sync = f'{url_remote}api/sync/delete_generated_files/{table}'
try:
    response = requests.get(url_sync, timeout=15)
    response.raise_for_status()
    sync_response = response.json()
    print("✅ Archivos generados eliminados en el servidor remoto:")
    print(sync_response)
except requests.exceptions.RequestException as e:
    print(f"❌ Error al eliminar archivos generados en el servidor remoto: {e}")

✅ Archivos generados eliminados en el servidor remoto:
{'status': 'success', 'message': '✅ Eliminación completa: 5 archivo(s) eliminado(s)', 'deleted_files': 5, 'failed_files': [], 'table_name': 'items'}


In [12]:
# Ejecutar localmente la sincronización de la tabla

url_local_sync = f'{local_url}/api/sync/sync_table/{table}'
print(url_local_sync)

try:
    response = requests.get(url_local_sync, timeout=15)
    response.raise_for_status()  # Verifica que la solicitud fue exitosa

    sync_response = response.json()
    print("✅ Respuesta de sincronización local:")
    print(sync_response)
except requests.exceptions.RequestException as e:
    print(f"❌ Error al sincronizar localmente: {e}")

http://localhost/accn//api/sync/sync_table/items
✅ Respuesta de sincronización local:
{'status': 'success', 'message': '✅ Sincronización completada para items', 'inserted': 0, 'updated': 89, 'table_name': 'items'}
